# 1. Load the libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import cv2
import glob
import numpy as np
import pandas as pd  
from PIL import Image
from skimage import measure
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import tensorflow as tf
from skimage.measure import label, regionprops, regionprops_table

# 2. ResNet50

## 2.1 Load the model

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model


#load pre-trained model
#model = ResNet50(include_top=False, weights="imagenet", input_shape=(50, 50, 3))
#model.summary()


#load trained model with binary classification
model = load_model("/home/gauss/Desktop/Training_ONLYTOPCLASSIFICATION_pooled_wells_d2d3d4/Trained_Model")
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                          

## 2.2 Assign Feature extraction layer

In [3]:
'''
#feature extraction from pre-trained model
layer = model.get_layer(name="conv5_block3_2_conv").output
output = GlobalAveragePooling2D()(layer)
# define new model
feature_extraction_model = Model(inputs=model.inputs, outputs=output)
# summarize
feature_extraction_model.summary()
'''


# My `trained_model` model
dense_layer_output = model.get_layer(name="dense_1").output
# Create a feature extraction model
feature_extraction_model = tf.keras.Model(inputs=model.input, outputs=dense_layer_output)
feature_extraction_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50_input (InputLayer)  [(None, 50, 50, 3)]      0         
                                                                 
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                             

In [4]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'


#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])


ch1 = []
ch2 = []
ch4 = []

for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+1])
    CH4_path = os.path.join(image_path, images[i+2])

    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1.append(np.array(ch1_img))
    ch2.append(np.array(ch2_img))
    ch4.append(np.array(ch4_img))

ch1_max = np.max(ch1)
ch2_max = np.max(ch2)
ch4_max = np.max(ch4)
print("Maximum intensity for channel 1:", ch1_max)
print("Maximum intensity for channel 2:", ch2_max)
print("Maximum intensity for channel 4:", ch4_max)

ch1_q099 =  np.quantile(ch1, 0.99)
ch2_q099 =  np.quantile(ch2, 0.99)
ch4_q099 =  np.quantile(ch4, 0.99)
print("Quantile_099 channel 1:", ch1_q099)
print("Quantile_099 channel 2:", ch2_q099)
print("Quantile_099 channel 4:", ch4_q099)

Maximum intensity for channel 1: 65314
Maximum intensity for channel 2: 65322
Maximum intensity for channel 4: 62641
Quantile_099 channel 1: 1848.0
Quantile_099 channel 2: 5207.0
Quantile_099 channel 4: 2457.0


## 2.3 Read images as tensors and extract features

In [5]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N8863__2020-04-08T11_42_16-Measurement 1'



#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

# "mean_feature_list" and "j" variables are initialized to empty list and 0, respectively to store the features for each image and to iterate over the list of masks.
mean_fetaure_list =[]
image_list = []
j=0

#loop with step size of 3 to iterate over a range of indices, pulling the filenames for three channels of each image.
for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+1])
    CH4_path = os.path.join(image_path, images[i+2])
    
    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    image_array_ch1 = np.array(ch1_img)
    image_array_ch2 = np.array(ch2_img)
    image_array_ch4 = np.array(ch4_img)


    #use cellpose mask to extract individual cell regions,to generate 50x50 pixel images.
    cellpose_path = os.path.join(mask_path, cellpose_mask[j])
    cellpose_img = Image.open(cellpose_path)
    masks_ch2 = np.array(cellpose_img)
    
    labels_ch2 = label(masks_ch2)
    props = regionprops(labels_ch2)
    
    j=j+1
    single_cell_bbox = []
    
    for cell_label, target_region in enumerate(props, start=1):
        centroid_row, centroid_col = target_region.centroid

        half_size = 25
        min_row = int(max(centroid_row - half_size, 0))
        max_row = int(min(centroid_row + half_size, image_array_ch2.shape[0]))
        min_col = int(max(centroid_col - half_size, 0))
        max_col = int(min(centroid_col + half_size, image_array_ch2.shape[1]))

        if (min_row > 0 and max_row < image_array_ch2.shape[0] and 
        min_col > 0 and max_col < image_array_ch2.shape[1]):

            target_cell_mask = labels_ch2 == cell_label

            cell_area_original = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch2[min_row:max_row, min_col:max_col], 0)

            empty_array_ch2 = np.zeros((50, 50))
            paste_row_original = (50 - cell_area_original.shape[0]) // 2
            paste_col_original = (50 - cell_area_original.shape[1]) // 2

            empty_array_ch2[paste_row_original:paste_row_original+cell_area_original.shape[0], paste_col_original:paste_col_original+cell_area_original.shape[1]] = cell_area_original

            empty_array_cellpose = np.zeros((50, 50))
            cell_area_cellpose = target_cell_mask[min_row:max_row, min_col:max_col]

            paste_row_cellpose = (50 - cell_area_cellpose.shape[0]) // 2
            paste_col_cellpose = (50 - cell_area_cellpose.shape[1]) // 2

            empty_array_cellpose[paste_row_cellpose:paste_row_cellpose+cell_area_cellpose.shape[0], paste_col_cellpose:paste_col_cellpose+cell_area_cellpose.shape[1]] = cell_area_cellpose

            cell_area_ch1 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch1[min_row:max_row, min_col:max_col], 0)
            cell_area_ch4 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch4[min_row:max_row, min_col:max_col], 0)

            empty_array_ch1 = np.zeros((50, 50))
            paste_row_ch1 = (50 - cell_area_ch1.shape[0]) // 2
            paste_col_ch1 = (50 - cell_area_ch1.shape[1]) // 2

            empty_array_ch4 = np.zeros((50, 50))
            paste_row_ch4 = (50 - cell_area_ch4.shape[0]) // 2
            paste_col_ch4 = (50 - cell_area_ch4.shape[1]) // 2

            empty_array_ch1[paste_row_ch1:paste_row_ch1+cell_area_ch1.shape[0], paste_col_ch1:paste_col_ch1+cell_area_ch1.shape[1]] = cell_area_ch1
            empty_array_ch4[paste_row_ch4:paste_row_ch4+cell_area_ch4.shape[0], paste_col_ch4:paste_col_ch4+cell_area_ch4.shape[1]] = cell_area_ch4
            
            '''
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10, 5))
            ax1.imshow(empty_array_cellpose, cmap='gray')
            ax1.set_title(f'Cellpose Cell {cell_label}')
            ax2.imshow(empty_array_ch2, cmap='gray')
            ax2.set_title(f'Channel 2 Cell {cell_label}')
            ax3.imshow(empty_array_ch1, cmap='gray')
            ax3.set_title(f'Channel 1 Cell {cell_label}')
            ax4.imshow(empty_array_ch4, cmap='gray')
            ax4.set_title(f'Channel 4 Cell {cell_label}')
            plt.show()
            '''
            
            empty_array_ch1 = (empty_array_ch1.astype('float32')/ch1_q099)*255
            empty_array_ch2 = (empty_array_ch2.astype('float32')/ch2_q099)*255
            empty_array_ch4 = (empty_array_ch4.astype('float32')/ch4_q099)*255

            singlecell = np.stack((empty_array_ch1, empty_array_ch2, empty_array_ch4), axis=-1)
            reshaped_singlecell = singlecell.reshape((1,50,50,3))
            single_cell_bbox.append(reshaped_singlecell)
            #print(np.max(empty_array_ch1))
            
    single_cells = np.array(single_cell_bbox)

    if len(single_cells) != 0:
        
        print(images[i])
    
        #convert single cell objects into a tensor using tf.data.Dataset.from_tensor_slices.
        single_cell_tensor = tf.data.Dataset.from_tensor_slices(single_cells)

        def extract_features(image):
            features = feature_extraction_model(image)
            return features

        #A map operation is used to apply a feature extraction function, extract_features, to each cell image 
        #using a pre-trained model. This creates a feature_dataset of extracted features.
        feature_dataset = single_cell_tensor.map(extract_features)

        #features are summed across all cells, divided by the number of cells, and added to the mean_feature_list.
        sum_feature_dataset = tf.zeros((1,512))

        for f in feature_dataset:
            sum_feature_dataset += f

        mean_feature = sum_feature_dataset/len(feature_dataset)

        mean_fetaure_list.append(mean_feature)
        image_list.append(images[i])

r01c01ch1.tiff
r01c02ch1.tiff
r01c03ch1.tiff
r01c04ch1.tiff
r01c05ch1.tiff
r01c06ch1.tiff
r01c07ch1.tiff
r01c08ch1.tiff
r01c09ch1.tiff
r01c10ch1.tiff
r01c11ch1.tiff
r01c12ch1.tiff
r01c13ch1.tiff
r01c14ch1.tiff
r01c15ch1.tiff
r01c16ch1.tiff
r01c17ch1.tiff
r01c18ch1.tiff
r01c19ch1.tiff
r01c20ch1.tiff
r01c21ch1.tiff
r01c22ch1.tiff
r01c23ch1.tiff
r01c24ch1.tiff
r02c01ch1.tiff
r02c02ch1.tiff
r02c03ch1.tiff
r02c04ch1.tiff
r02c05ch1.tiff
r02c06ch1.tiff
r02c07ch1.tiff
r02c08ch1.tiff
r02c09ch1.tiff
r02c10ch1.tiff
r02c11ch1.tiff
r02c12ch1.tiff
r02c13ch1.tiff
r02c14ch1.tiff
r02c15ch1.tiff
r02c17ch1.tiff
r02c18ch1.tiff
r02c19ch1.tiff
r02c20ch1.tiff
r02c21ch1.tiff
r02c22ch1.tiff
r02c23ch1.tiff
r02c24ch1.tiff
r03c01ch1.tiff
r03c02ch1.tiff
r03c03ch1.tiff
r03c04ch1.tiff
r03c05ch1.tiff
r03c06ch1.tiff
r03c07ch1.tiff
r03c08ch1.tiff
r03c09ch1.tiff
r03c10ch1.tiff
r03c11ch1.tiff
r03c12ch1.tiff
r03c13ch1.tiff
r03c14ch1.tiff
r03c15ch1.tiff
r03c16ch1.tiff
r03c17ch1.tiff
r03c18ch1.tiff
r03c19ch1.tiff
r03c20ch1.

In [6]:
#Convert mean feature tensor array to numpy array
feature_list = [tensor.numpy() for tensor in mean_fetaure_list]

In [7]:
len(feature_list)

381

In [8]:
# Create a 2D numpy array from the feature list
feature_array = np.vstack(feature_list)

# Create a DataFrame with column names
df = pd.DataFrame(feature_array, columns=['feature{}'.format(i+1) for i in range(512)])

#add col to the beginning of the dataframe
df.insert(0, 'Image', range(1,len(feature_list)+1))
df

,Image,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature503,feature504,feature505,feature506,feature507,feature508,feature509,feature510,feature511,feature512
0,1,0.047953,0.798119,0.492550,0.093670,0.646187,0.430536,0.299065,0.110803,0.332921,...,0.258722,0.389015,0.411697,0.635522,0.434083,0.094881,0.589774,0.049330,0.151310,0.660716
1,2,0.077034,0.892100,0.371673,0.114407,0.757134,0.610607,0.144242,0.117551,0.345691,...,0.172886,0.512372,0.723745,0.709574,0.269752,0.068852,0.594077,0.066310,0.152505,0.337240
2,3,0.012620,0.719137,0.232990,0.056743,0.666103,0.270240,0.481965,0.034168,0.174311,...,0.065795,0.587439,0.381682,0.803182,0.559407,0.006726,0.861949,0.010619,0.056939,0.466257
3,4,0.046719,0.901616,0.493199,0.105442,0.649296,0.699164,0.178024,0.103344,0.376702,...,0.086613,0.617177,0.601822,0.783579,0.339774,0.047493,0.487531,0.071338,0.193543,0.386413
4,5,0.080421,0.782797,0.468598,0.086466,0.637868,0.376176,0.377017,0.119080,0.333277,...,0.241602,0.414476,0.385331,0.628729,0.433223,0.049952,0.539621,0.066424,0.187218,0.743284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,377,0.057030,0.790024,0.460898,0.068438,0.671361,0.392791,0.342382,0.091483,0.309411,...,0.242168,0.403899,0.395866,0.620602,0.431728,0.060095,0.538253,0.039741,0.140531,0.737891
377,378,0.078792,0.822935,0.488574,0.095452,0.630885,0.439699,0.337943,0.132242,0.384476,...,0.268315,0.387881,0.401267,0.593000,0.451507,0.076386,0.481037,0.065316,0.184563,0.759256
378,379,0.088978,0.799516,0.471923,0.090062,0.639507,0.417336,0.344172,0.126720,0.371090,...,0.256954,0.391619,0.387199,0.608164,0.449996,0.067028,0.497704,0.057815,0.170717,0.758251
379,380,0.072292,0.802108,0.461153,0.084595,0.646976,0.424353,0.322845,0.121341,0.345379,...,0.257803,0.397284,0.407772,0.618258,0.443128,0.071876,0.506222,0.052625,0.158915,0.745348


In [9]:
#save dataframe as excel 
df.to_excel("/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/MHB/Plate_8.xlsx")


In [10]:
len(image_list)

381

In [ ]:
df

In [ ]:
image_list[50:150]